# RNA-Seq analysis of T24, T1, T0 samples 


In [1]:
suppressMessages(library(DESeq2))
suppressMessages(library(ggplot2))
suppressMessages(library(cowplot))

In [2]:
cbbPalette <- c("#999999", "#E69F00", "#56B4E9", "#009E73", "#F0E442", "#0072B2", "#D55E00", "#CC79A7")


In [3]:
annotations <- read.table(file.path('..', 'data', 'annotations', 'hg38_gene_names_stripped.tsv'), 
                          header=F, 
                          col.names = c('gene_id', 'gene_name', 'gene_type'))
rownames(annotations) <- annotations$gene_id

readcounts.dir <- file.path('..', 'data', 'read_counts', 'byExon', 'RNA_seq')
results.dir <- file.path('..', 'results', 'RNA_seq')
design.file <-  file.path('..', 'data', 'design_files', 'rna_seq_design.tsv')

## Suffix of htseq-count output
counts.suffix <- '.exon.counts.tsv'

write_results <- function(df, prefix){
  df<- as.data.frame(df)
  df <- df[order(df$padj),]
  df$gene_name <- annotations[rownames(df),]$gene_name
  df.sig <- subset(df, padj<0.05)
  df.sig.up <- subset(df.sig, log2FoldChange>0)
  df.sig.down <- subset(df.sig, log2FoldChange<0)
  write.table(df, file = file.path(results.dir, 
                                   paste(prefix, 'tsv', sep='.')), sep = '\t')

  write.table(df.sig, file = file.path(results.dir, 
                                   paste(prefix, 'sig', 'tsv', sep='.')), sep = '\t')
  write.table(df.sig.up,  file = file.path(results.dir,
                                       paste(prefix, 'sig', 'up', 'tsv', sep='.')), sep = '\t')
  write.table(df.sig.down,  file = file.path(results.dir,
                                         paste(prefix, 'sig', 'down', 'tsv', sep='.')), sep = '\t')
  return (df.sig)
}

PCAplot <- function(rld){
  data <- plotPCA(rld, intgroup=c('Cell.line', 'Time'), returnData=TRUE, ntop=1210000)
  percentVar <- round(100 * attr(data, 'percentVar'))
  ggplot(data, aes(PC1, PC2, color=Time, shape=CellLine)) +
    geom_point(size=3) +
    theme_bw(base_size = 16) + 
    xlab(paste0('PC1: ',percentVar[1],'% variance')) +
    ylab(paste0('PC2: ',percentVar[2],'% variance')) +
    coord_fixed() +
    scale_color_manual("Time", values = c('T0' = cbbPalette[6], 
                                          'T1' = cbbPalette[7],                                         
                                          'T24' = cbbPalette[4])) +
    theme(aspect.ratio=1, plot.title = element_text(hjust = 0.5))
}

PCAplot.cellLine <- function(rld){
  data <- plotPCA(rld, intgroup=c('Time'), returnData=TRUE)
  percentVar <- round(100 * attr(data, 'percentVar'))
  ggplot(data, aes(PC1, PC2, color=Time)) +
    geom_point(size=3) +
    theme_bw(base_size = 16) + 
    xlab(paste0('PC1: ',percentVar[1],'% variance')) +
    ylab(paste0('PC2: ',percentVar[2],'% variance')) +
    coord_fixed() +
    scale_color_manual("Time", values = c('T0' = cbbPalette[6], 
                                          'T1' = cbbPalette[7], 
                                          'T24' = cbbPalette[4])) + 
    theme(aspect.ratio=1, plot.title = element_text(hjust = 0.5))
}


design.info <- read.table(design.file, header=T, stringsAsFactors=FALSE)
design.info$Time <- as.factor(design.info$Time)
design.info$Cell.line <- as.factor(design.info$Cell_line)
files <- paste(design.info$SampleFile, counts.suffix, sep='')
sampleName <- design.info$SampleName
Time <- design.info$Time
Cell.line <- design.info$Cell.line
group <- factor(paste0(Cell.line, '_', Time))
sampleTable <- data.frame(sampleName = sampleName, 
                          fileName = files, 
                          Cell.line = Cell.line,  
                          Time = Time, 
                          group = group)
ddsHTSeq <- DESeqDataSetFromHTSeqCount(sampleTable = sampleTable, 
                                       directory = readcounts.dir,
                                       design = ~ group)
rownames(ddsHTSeq) <- gsub('\\.[0-9]+', '', rownames(ddsHTSeq))
ddsHTSeq <- ddsHTSeq[ rowSums(counts(ddsHTSeq)) > 1, ]
dds <- DESeq(ddsHTSeq)

resultsNames(dds)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


[1] "Intercept"                 "group_U251_T1_vs_U251_T0" 
[3] "group_U251_T24_vs_U251_T0" "group_U343_T0_vs_U251_T0" 
[5] "group_U343_T1_vs_U251_T0"  "group_U343_T24_vs_U251_T0"

# All T1 vs T0

In [4]:
All.res.T1vsT0 <- results(dds, 
                          contrast = list(c('group_U251_T1_vs_U251_T0', 
                                            'group_U343_T1_vs_U251_T0'), 
                                          'group_U343_T0_vs_U251_T0'), 
                          listValues = c(1, -1))
write_results(All.res.T1vsT0, 'All.RNASeq.res.T1vsT0')

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene_name
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
ENSG00000162783,3859.0501,1.1523116,0.14304024,8.055857,7.892411e-16,1.185282e-11,IER5
ENSG00000196670,879.1671,-1.1075250,0.16077484,-6.888671,5.631612e-12,4.228778e-08,ZFP62
ENSG00000125740,127.1540,2.1548858,0.32128571,6.707070,1.985710e-11,9.564706e-08,FOSB
ENSG00000181026,6536.5288,0.9609583,0.14405866,6.670604,2.547531e-11,9.564706e-08,AEN
ENSG00000081019,1250.8434,-1.0362656,0.16010273,-6.472504,9.639176e-11,2.301739e-07,RSBN1
ENSG00000101665,299.3777,1.7419188,0.26984371,6.455288,1.080134e-10,2.301739e-07,SMAD7
ENSG00000155090,2370.5774,1.0224928,0.15819502,6.463495,1.023116e-10,2.301739e-07,KLF10
ENSG00000196263,349.2331,-1.4600091,0.22684817,-6.436063,1.226123e-10,2.301739e-07,ZNF471
ENSG00000164011,428.0047,-0.9660298,0.15594801,-6.194563,5.844698e-10,9.752853e-07,ZNF691


# All T24 vs T0

In [5]:
All.res.T24vsT0 <- results(dds, 
                           contrast = list(c('group_U251_T24_vs_U251.T0', 
                                             'group_U343_T24_vs_U251.T0'), 
                                           'group_U343_T0_vs_U251.T0'), 
                           listValues = c(1, -1))
write_results(All.res.T24vsT0, 'All.RNASeq.res.T24vsT0')

ERROR: Error in checkContrast(contrast, resNames): all elements of the contrast as a list of length 2 should be elements of 'resultsNames(object)'


# All T24 vs T1

In [ ]:
All.res.T24vsT1 <- results(dds, 
                           contrast = list(c('group_U251_T24_vs_U251.T0', 
                                             'group_U343_T24_vs_U251.T0'), 
                                           c('group_U251_T1_vs_U251.T0',
                                             'group_U343_T1_vs_U251.T0')), 
                           listValues = c(1, -1))
write_results(All.res.T24vsT1, 'All.RNASeq.res.T24vsT1')